In [ ]:
# # Nom du fichier source et du fichier de sortie
# fichier_entree = "train.txt"
# fichier_sortie = "train_cut.txt"

# # Lecture du fichier et filtrage
# with open(fichier_entree, "r", encoding="utf-8") as f_in:
#     lignes = f_in.readlines()

# # Garder une ligne sur deux
# lignes_filtrees = lignes[::2]  # [::2] = toutes les 2 lignes en partant de 0

# # Écriture du résultat dans un nouveau fichier
# with open(fichier_sortie, "w", encoding="utf-8") as f_out:
#     f_out.writelines(lignes_filtrees)

# print(f"{len(lignes_filtrees)} lignes conservées dans '{fichier_sortie}'.")


entrainer modèle pour classifier :
    -classification personelle
- associer les phrases à des cas féminins masculins ou neutre (classes)
ensuite : effacer les noms et voir si ça marche tjr
    => test de prédiction

- prendre des sujets : ex : faire un régime
    voir les thématiques associées à des femmes
    quand je fait la détection, ne pas utiliser les noms propres différents (NOM1)

    ensuite, inverser nom féminin et masculin et tester la réussite du schéma winogrande
    1 nom féminin, masculin  ou les 2

environ 200-400 annotation
éventuellement utiliser des recherches de mots (pas trop généraux et plusieurs)

utiliser : sklearn => word embeding (BERT)

ensuite : validation croisée , couper le corpus en pluseurs morceaux et faire le test plusieurs fois, car on manque de données
=> k fold

faire des stats sur les sous ensemble d'annotation

prendre RoBerta finetuné sur winograde et lui proposer de items non biaisé à résoudre et comparer les scores avecdes items biaisés

pour bert: faire en sorte que ça donne bien les embedding

In [33]:
from google.colab import files
uploaded = files.upload()


Saving class_2.csv to class_2 (3).csv


sport
caring
wealth
advice

hesitation, question, advice
performance, sucess, competence
Failure
comparison of performance, road to sucess
relationship, helping

aussi : comptabiliser her/his pour les femmes/h

In [40]:
df

,texte,succes,embedding
0,Ian volunteered to eat Dennis's menudo after a...,0.0,"[-0.052102488, 0.23901948, 0.10077491, -0.0224..."
1,"He never comes to my home, but I always go to ...",1.0,"[-0.011725236, 0.30165508, 0.10036195, -0.0942..."
2,"Kyle doesn't wear leg warmers to bed, while Lo...",0.0,"[-0.013384739, 0.229838, 0.07188911, -0.023034..."
3,The GPS and map helped me navigate home. I go...,0.0,"[-0.042121787, 0.26607785, 0.118360154, -0.049..."
4,Emily looked up and saw Patricia racing by ove...,0.0,"[-0.0050911666, 0.24380973, 0.14531961, -0.074..."
...,...,...,...
196,"Jessica often experiences severe nausea, Victo...",1.0,"[-0.038923547, 0.31035942, 0.10008639, -0.0708..."
197,The system that Jeffrey used was amazing compa...,1.0,"[0.017917642, 0.1726549, 0.05963793, -0.088282..."
198,It took Jerry longer to shampoo his hair than ...,0.0,"[0.013828062, 0.24519454, 0.08045871, -0.07406..."
199,Have you ever been on a nature walk Samantha ...,0.0,"[0.003861581, 0.29913023, 0.10493905, -0.06287..."


In [64]:
import pandas as pd

# Charger le fichier CSV
df_original = pd.read_csv("class_2.csv")  # Remplace par le nom réel

def extract_sentence(text_string):
    # Regex pour trouver 'sentence ' suivi de n'importe quel caractère (.*?)
    # jusqu'à la prochaine virgule et l'espace ', ' (qui précède généralement 'option1' ou d'autres clés)
    # ou jusqu'à la fin de la chaîne si rien ne suit.
    match = re.search(r'sentence\s(.*?)(?:,\soption1|, answer|\})$', text_string)

    if match:
        return match.group(1).strip() # .strip() pour supprimer les espaces blancs en début/fin
    return None # Retourne None si la phrase n'est pas trouvée (cas d'erreur ou format inattendu)


sentiment_mapping = {
    'hesitation, question, advice': 0,
    'comparison of performance, road to sucess': 0,
    'other': 0,
    'Failure': 0,
    'relationship, helping' : 1,
    'performance, sucess, competence' : 0,
    'nan' : 0,
    # Ajoutez d'autres sentiments si vous en avez, et mappez-les à 0 ou 1
}

df_original['succes'] = df_original['sentiment'].map(sentiment_mapping)
df_original['clean_text'] = df_original['text'].apply(extract_sentence)
df_original['succes'].fillna(0, inplace=True)

# Option B: Si vous voulez que SEULEMENT un sentiment spécifique (par exemple 'succes') soit 1, et tout le reste 0
# Décommentez et utilisez cette ligne à la place de l'Option A si c'est votre cas
# df_original['succes'] = (df_original['sentiment'] == 'succes').astype(int)

# --- Conversion en dictionnaire du format désiré ---
data = {
    'texte': df_original['clean_text'].tolist(),
    'succes': df_original['succes'].tolist()
}



print("Dictionnaire converti :")
# Affichons les premières entrées pour vérifier
print("Texte (premières 5 entrées) :", data['texte'][:5])
print("Succès (premières 5 entrées) :", data['succes'][:5])
print(f"Nombre total de phrases : {len(data['texte'])}")
print(f"Nombre total d'étiquettes : {len(data['succes'])}")

# Vous pouvez maintenant utiliser ce dictionnaire 'data' pour créer votre DataFrame Pandas
# comme dans les exemples précédents pour l'entraînement.
df_for_training = pd.DataFrame(data)
print("\nDataFrame pour l'entraînement (à partir du dictionnaire) :")
print(df_for_training.head())

Dictionnaire converti :
Texte (premières 5 entrées) : ["Ian volunteered to eat Dennis's menudo after already having a bowl because _ despised eating intestine., option1 Ian, option2 Dennis, answer 2", 'He never comes to my home, but I always go to his house because the _ is smaller., option1 home, option2 house, answer 1', "Kyle doesn't wear leg warmers to bed, while Logan almost always does. _ is more likely to live in a colder climate., option1 Kyle, option2 Logan, answer 2", 'The GPS and map helped me navigate home.  I got lost when the _ got turned upside down., option1 GPS, option2 map, answer 2', 'Emily looked up and saw Patricia racing by overhead, as _ was on the ramp ., option1 Emily, option2 Patricia, answer 2']
Succès (premières 5 entrées) : [0.0, 0.0, 0.0, 0.0, 0.0]
Nombre total de phrases : 201
Nombre total d'étiquettes : 201

DataFrame pour l'entraînement (à partir du dictionnaire) :
                                               texte  succes
0  Ian volunteered to eat De

<ipython-input-64-2998114979>:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_original['succes'].fillna(0, inplace=True)


In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
# ATTENTION: Changer CamembertTokenizer et CamembertModel par BertTokenizer et BertModel
from transformers import BertTokenizer, BertModel # <--- CHANGEMENT ICI
import torch
import numpy as np
import re

# --- 1. Préparation des données (Maintenant en ANGLAIS) ---
# Il est crucial que vos données d'entraînement soient en anglais si vous utilisez un BERT anglais.
# df = pd.DataFrame(data)
# data = {
#     'texte': [ # Le nom de la colonne reste 'texte' pour la cohérence avec le reste du code
#         "i like building a new house", "He created a new machine", "building a bigger flat", "She increased the size of the field", "he eated a huge portion", "i like thanksgiving"
#     ],
#     'succes': [1, 1, 1, 1, 0, 0,]
# }

df = pd.DataFrame(data)

# Renommer la colonne pour la cohérence si vous avez des habitudes d'autres frameworks
# Mais pour cette approche, cela n'est pas strictement nécessaire pour la régression logistique
# df = df.rename(columns={'succes': 'labels'})

print("Aperçu de notre jeu de données (maintenant en anglais) :")
print(df)
print("-" * 50)

# 2. Définition du périphérique (CPU ou GPU)
# C'est important pour les calculs de BERT
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Utilisation du périphérique : {device}")
print("-" * 50)

# 3. Chargement du tokenizer et du MODÈLE BERT (sans tête de classification)
# Nous chargeons BertModel, et non BertForSequenceClassification
# car nous voulons juste les embeddings des tokens, pas la couche de classification.
# MODIFICATION: Utiliser 'bert-base-uncased' ou 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # <--- CHANGEMENT ICI
model = BertModel.from_pretrained('bert-base-uncased')         # <--- CHANGEMENT ICI
model.to(device) # Envoyer le modèle sur le GPU si disponible
model.eval()     # Mettre le modèle en mode évaluation (désactive dropout, etc.)

print("Tokenizer et modèle BERT chargés (pour l'anglais) et mis en mode évaluation.")
print("-" * 50)

# 4. Fonction pour obtenir les embeddings des phrases
def get_sentence_embedding(text):
    # Ajouter les tokens spéciaux de BERT et encoder la phrase
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

    with torch.no_grad(): # Désactiver le calcul des gradients (pas d'entraînement ici)
        outputs = model(**inputs)

    # Récupérer l'embedding du token [CLS] (le premier token)
    # L'embedding du token [CLS] est souvent utilisé comme la représentation globale de la phrase
    # Il est à la position 0 du last_hidden_state
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    return cls_embedding

# Appliquer la fonction pour obtenir les embeddings pour toutes les phrases
# Attention: Avec un très grand dataset, cette étape peut prendre du temps
# et nécessiter une gestion des batchs pour la mémoire GPU.
df['embedding'] = df['texte'].apply(get_sentence_embedding)

# X est maintenant la matrice des embeddings, y est la cible
X = np.stack(df['embedding'].tolist()) # Convertir la liste de tableaux numpy en une seule matrice
y = df['succes']

print(f"Dimension des embeddings : {X.shape}") # (nombre de phrases, dimension de l'embedding, e.g., 768)
print("-" * 50)

# 5. Division des données (embeddings et labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"Taille de l'ensemble d'entraînement (embeddings) : {X_train.shape[0]} échantillons")
print(f"Taille de l'ensemble de test (embeddings) : {X_test.shape[0]} échantillons")
print("-" * 50)

# 6. Entraînement de la Régression Logistique sur les embeddings de BERT
log_reg_model = LogisticRegression(max_iter=1000, random_state=42)
log_reg_model.fit(X_train, y_train)

print("Modèle de Régression Logistique entraîné sur les embeddings de BERT (anglais).")
print("-" * 50)

# 7. Évaluation du modèle de Régression Logistique
y_pred = log_reg_model.predict(X_test)

print("Rapport de classification de la Régression Logistique :")
print(classification_report(y_test, y_pred, target_names=['Non-succès', 'Succès']))

print("Matrice de confusion de la Régression Logistique :")
print(confusion_matrix(y_test, y_pred))
print("-" * 50)

# 8. Prédiction sur de nouvelles phrases (exemples)
new_sentences = [
    "Kayla dated many more people at once than Betty, fnjfkvngjeksnvgjknresvngesnvgjrens:vgjsenvgkwas in an open .",
]

print("Prédictions sur de nouvelles phrases avec BERT (anglais) + Régression Logistique :")
for sentence in new_sentences:
    # 1. Obtenir l'embedding de la phrase avec BERT
    sentence_embedding = get_sentence_embedding(sentence)

    # 2. Reshape l'embedding pour qu'il soit un échantillon unique (1, dimension)
    # car .predict() attend une entrée 2D
    sentence_embedding_reshaped = sentence_embedding.reshape(1, -1)

    # 3. Faire la prédiction avec le modèle de régression logistique
    prediction = log_reg_model.predict(sentence_embedding_reshaped)[0]
    probabilities = log_reg_model.predict_proba(sentence_embedding_reshaped)[0]

    prediction_label = "Success" if prediction == 1 else "Non-success" # CHANGEMENT dans le label pour l'affichage
    prob_succes = probabilities[1] # Probabilité de la classe 1 (Succès)

    print(f"Sentence : '{sentence}' -> Prediction : {prediction_label} (Success Probability : {prob_succes:.2f})")

Aperçu de notre jeu de données (maintenant en anglais) :
                                                 texte  succes
0    Ian volunteered to eat Dennis's menudo after a...     0.0
1    He never comes to my home, but I always go to ...     0.0
2    Kyle doesn't wear leg warmers to bed, while Lo...     0.0
3    The GPS and map helped me navigate home.  I go...     0.0
4    Emily looked up and saw Patricia racing by ove...     0.0
..                                                 ...     ...
196  Jessica often experiences severe nausea, Victo...     0.0
197  The system that Jeffrey used was amazing compa...     0.0
198  It took Jerry longer to shampoo his hair than ...     0.0
199  Have you ever been on  a nature walk Samantha ...     0.0
200  Ben was choosing between a clothing business a...     0.0

[201 rows x 2 columns]
--------------------------------------------------
Utilisation du périphérique : cuda
--------------------------------------------------
Tokenizer et modèle BERT c

from logging import raiseExceptions
vrai positifs et négatifs
check : prendre plein des fois la même phrase et voir si c'est bien reconnu
faire un graphe de la probabilité en fonction des différentes itéraiseExceptions


est ce que c'est possible de faire dans l'autre sens ? => partir des embeddings moyens => et arriver à des catégories
à grande dimension, on peut mettre un nb exp de vecteurs orthogonaux
anisotropie => ragarder papier,
on a des connes : car faire des sommes d'objets dans tous les espaces => n'encode rien


faire un classifier pour les femmes et les hommes
voir les corrélations des dimmensions avec le genre
mettre un couche linéaire : 768 dimension => score


réunion vendredi mitn llm : rammener des questions de recherche et des hypothèses
voir le vocabulaire très genré, ex : diet
faire des permutations des prénomask

utiliser she ou he
refaire tourner


éviter la comparaison : car c'est trop syntaxique
prendre des sujets plus concrets
bricolage
scolaire
